In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
sns.set()
sns.set_context('talk')

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 104)
import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

In [2]:
df = pd.read_csv('TX_2016.csv')
df

,Unnamed: 0,Crash Type,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,Single Vehicle,NaN,NaN,Multiple Vehicles,NaN,NaN,Total,NaN,NaN
1,FieldDesc,Number,Alcohol--Related,Percent Alcohol--Related,Number1,Alcohol--Related1,Percent Alcohol--Related1,Number2,Alcohol--Related2,Percent Alcohol--Related2
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Midnight to 2:59 a.m.,351,213,61,162,111,68,513,324,63
4,3 a.m. to 5:59 a.m.,229,111,49,133,81,61,362,192,53
5,6 a.m. to 8:59 a.m.,161,47,29,194,30,15,355,77,22
6,9 a.m. to 11:59 a.m.,132,19,14,156,28,18,288,47,16
7,Noon to 2:59 p.m.,165,23,14,239,48,20,404,71,18
8,3 p.m. to 5:59 p.m.,176,49,28,241,75,31,417,125,30
9,6 p.m. to 8:59 p.m.,322,122,38,217,103,47,539,225,42


In [3]:
print(df.columns)

Index(['Unnamed: 0', 'Crash Type', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'],
      dtype='object')


In [4]:
df.columns = ['Time Frame', 'Single Vehicles', 'Alcohol Related' , 'Alcohol Percentage', 'Multiple Vehicles','Alcohol Related', 'Alcohol Percentage' ,'Total Vehicles' ,'Total Alcohol Related', 'Total Alcohol Percentage']




In [5]:
df

,Time Frame,Single Vehicles,Alcohol Related,Alcohol Percentage,Multiple Vehicles,Alcohol Related,Alcohol Percentage,Total Vehicles,Total Alcohol Related,Total Alcohol Percentage
0,NaN,Single Vehicle,NaN,NaN,Multiple Vehicles,NaN,NaN,Total,NaN,NaN
1,FieldDesc,Number,Alcohol--Related,Percent Alcohol--Related,Number1,Alcohol--Related1,Percent Alcohol--Related1,Number2,Alcohol--Related2,Percent Alcohol--Related2
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Midnight to 2:59 a.m.,351,213,61,162,111,68,513,324,63
4,3 a.m. to 5:59 a.m.,229,111,49,133,81,61,362,192,53
5,6 a.m. to 8:59 a.m.,161,47,29,194,30,15,355,77,22
6,9 a.m. to 11:59 a.m.,132,19,14,156,28,18,288,47,16
7,Noon to 2:59 p.m.,165,23,14,239,48,20,404,71,18
8,3 p.m. to 5:59 p.m.,176,49,28,241,75,31,417,125,30
9,6 p.m. to 8:59 p.m.,322,122,38,217,103,47,539,225,42


In [6]:
df = df.iloc[3:]

In [7]:
df

,Time Frame,Single Vehicles,Alcohol Related,Alcohol Percentage,Multiple Vehicles,Alcohol Related,Alcohol Percentage,Total Vehicles,Total Alcohol Related,Total Alcohol Percentage
3,Midnight to 2:59 a.m.,351,213,61,162,111,68,513,324,63
4,3 a.m. to 5:59 a.m.,229,111,49,133,81,61,362,192,53
5,6 a.m. to 8:59 a.m.,161,47,29,194,30,15,355,77,22
6,9 a.m. to 11:59 a.m.,132,19,14,156,28,18,288,47,16
7,Noon to 2:59 p.m.,165,23,14,239,48,20,404,71,18
8,3 p.m. to 5:59 p.m.,176,49,28,241,75,31,417,125,30
9,6 p.m. to 8:59 p.m.,322,122,38,217,103,47,539,225,42
10,9 p.m. to 11:59 p.m.,356,152,43,193,126,65,549,278,51
11,Total,1892,736,39,1535,603,39,3427,1338,39


In [8]:
df.at[3,'Time Frame'] = '12:00 am - 3:00 am'
df.at[4,'Time Frame'] = '3:00 am - 6:00 am'
df.at[5,'Time Frame'] = '6:00 am - 9:00 am'
df.at[6,'Time Frame'] = '9:00 am - 12:00 pm'
df.at[7,'Time Frame'] = '12:00 pm - 3:00 pm'
df.at[8,'Time Frame'] = '3:00 pm - 6:00 pm'
df.at[9,'Time Frame'] = '6:00 pm - 9:00 pm'
df.at[10,'Time Frame'] = '9:00 pm - 12:00 am'



In [9]:
df

,Time Frame,Single Vehicles,Alcohol Related,Alcohol Percentage,Multiple Vehicles,Alcohol Related,Alcohol Percentage,Total Vehicles,Total Alcohol Related,Total Alcohol Percentage
3,12:00 am - 3:00 am,351,213,61,162,111,68,513,324,63
4,3:00 am - 6:00 am,229,111,49,133,81,61,362,192,53
5,6:00 am - 9:00 am,161,47,29,194,30,15,355,77,22
6,9:00 am - 12:00 pm,132,19,14,156,28,18,288,47,16
7,12:00 pm - 3:00 pm,165,23,14,239,48,20,404,71,18
8,3:00 pm - 6:00 pm,176,49,28,241,75,31,417,125,30
9,6:00 pm - 9:00 pm,322,122,38,217,103,47,539,225,42
10,9:00 pm - 12:00 am,356,152,43,193,126,65,549,278,51
11,Total,1892,736,39,1535,603,39,3427,1338,39


In [10]:
null_rows = df.isnull().any(axis = 1).sum()
null_rows

0